In [147]:
import pandas as pd
import plotly.graph_objects as go
import utils

In [148]:
def plot_candles(df_plot, df_markers = None):
    """  Create plot outputs, df_markers (markers/line) optional"""

    plot_colours_buy = ['#043ef9', '#eb5334', '#34eb37']
    plot_colours_sell = ['white', 'red', 'yellow']

    fig = go.Figure()
    
    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low = df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor="#24A06B",
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color="#2EC886",
        decreasing_line_color="#FF3A4C"
        ))


    if df_markers is not None:
        fig.add_trace(go.Candlestick(
        x=df_markers.time, open=df_markers.mid_o, high=df_markers.mid_h, low = df_markers.mid_l, close=df_markers.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor="#3480eb",
        decreasing_fillcolor="#3480eb",
        increasing_line_color="#3480eb",
        decreasing_line_color="#3480eb"
        ))        

    
    fig.update_layout(width=1000, height=400, paper_bgcolor = "#1e1e1e", plot_bgcolor = "#1e1e1e",
                    margin=dict(l=10, b=10, t=30, r=10), 
                    font=dict(size=10, color="#e1e1e1"),
                    title=f"Candle Chart - Blue Candle Indicators")
    fig.update_xaxes(gridcolor="#1f292f",
                    showgrid=True,
                    fixedrange=True,
                    rangeslider=dict(visible=False),
                    rangebreaks=[
                        dict(bounds=["sat", "mon"])
                        ]
                    )
    fig.update_yaxes(gridcolor="#1f292f",
                    showgrid=True)

    fig.show()

Note:  utils.get_histdf_fromcsv is new.  It, simply, returns DataFrame having parsed time "string" back to datetime format.  There are other - more efficient - ways we could do this...

In [149]:
pair = "USD_JPY"
granularity = "H4"
df_raw = utils.get_histdf_fromcsv(pair, granularity)
df_raw.shape

(4675, 15)

In [150]:
df_raw.columns

Index(['Unnamed: 0', 'time', 'volume', 'bid_o', 'bid_h', 'bid_l', 'bid_c',
       'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [151]:
cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']

In [152]:
df = df_raw[cols].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4675 entries, 0 to 4674
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   time    4675 non-null   datetime64[ns, tzutc()]
 1   mid_o   4675 non-null   float64                
 2   mid_h   4675 non-null   float64                
 3   mid_l   4675 non-null   float64                
 4   mid_c   4675 non-null   float64                
dtypes: datetime64[ns, tzutc()](1), float64(4)
memory usage: 182.7 KB


In [153]:
SMALL_BODY = 0.2
CLOSE_DISTANCE = 0.15
DOJI_BODY = 0.05
FULL_BODY = 0.95
CENTER_DISTANCE_HIGH = 0.55
CENTER_DISTANCE_LOW = 0.45
BIG_BODY = 0.6
ENGULFING_FACTOR = 1.1


def apply_top_end_distance(row):
    if row.DIRECTION == 1:
        return row.mid_h - row.mid_c
    else:
        return row.mid_h - row.mid_o

def apply_bottom_end_distance(row):
    if row.DIRECTION == 1:
        return row.mid_o - row.mid_l
    else:
        return row.mid_c - row.mid_l
    
def apply_hammer(row):
    if row.BODY_PERC < SMALL_BODY:
        if row.DIST_TOP_PERC < CLOSE_DISTANCE or row.DIST_BOTTOM_PERC < CLOSE_DISTANCE:
            return True
    return False

def apply_spinning_top(row):
    if row.BODY_PERC < SMALL_BODY:
        if row.DIST_TOP_PERC < CENTER_DISTANCE_HIGH and row.DIST_BOTTOM_PERC < CENTER_DISTANCE_LOW:
            return True
    return False

def apply_engulfing(row):
    if row.PREV_DIRECTION != row.DIRECTION:
        if row.PREV_BODY_PERC > BIG_BODY and row.BODY_PERC > BIG_BODY:
            if row.BODY_RANGE > row.PREV_BODY_RANGE * ENGULFING_FACTOR:
                return True
    return False



def apply_stats(df):
    df['RANGE'] = df.mid_h - df.mid_l
    df['BODY_RANGE'] = abs(df.mid_c - df.mid_o)
    df['CENTER'] = (df.mid_h - df.mid_l) / 2 + df.mid_l
    df['BODY_PERC'] =  df.BODY_RANGE / df.RANGE
    df['DIRECTION'] =  df.mid_c - df.mid_o
    df['DIRECTION'] = df['DIRECTION'].apply(lambda x: 1 if x >= 0 else -1)
    df['DIST_TOP'] = df.apply(apply_top_end_distance, axis = 1)
    df['DIST_BOTTOM'] = df.apply(apply_bottom_end_distance, axis = 1)
    df['DIST_TOP_PERC'] = df.DIST_TOP / df.RANGE
    df['DIST_BOTTOM_PERC'] = df.DIST_BOTTOM / df.RANGE
    df['PREV_BODY_RANGE'] = df.BODY_RANGE.shift(1)
    df['PREV_BODY_PERC'] = df.BODY_PERC.shift(1)
    df['PREV_DIRECTION'] = df.DIRECTION.shift(1)

    df.dropna(inplace=True)
    return df

def apply_patterns(df):
    df['HAMMER'] = df.apply(apply_hammer, axis = 1)
    df['SPINNING_TOP'] = df.apply(apply_hammer, axis = 1)
    df['DOJI'] = df['BODY_PERC'].apply(lambda x: True if x < DOJI_BODY else False)
    df['MARUBOZU'] = df['BODY_PERC'].apply(lambda x: True if x > FULL_BODY else False)
    df['ENGULFING'] = df.apply(apply_engulfing, axis = 1)

    return df



In [154]:
df = df_raw[cols].copy()
df = apply_stats(df)
df = apply_patterns(df)
# df = df.iloc[700:750]

In [155]:
df[df.DOJI==True].head()

,time,mid_o,mid_h,mid_l,mid_c,RANGE,BODY_RANGE,CENTER,BODY_PERC,DIRECTION,...,DIST_TOP_PERC,DIST_BOTTOM_PERC,PREV_BODY_RANGE,PREV_BODY_PERC,PREV_DIRECTION,HAMMER,SPINNING_TOP,DOJI,MARUBOZU,ENGULFING
33,2020-01-09 10:00:00+00:00,109.432,109.489,109.317,109.432,0.172,0.000,109.403,0.000000,1,...,0.331395,0.668605,0.156,0.857143,1.0,False,False,True,False,False
39,2020-01-10 10:00:00+00:00,109.643,109.688,109.504,109.652,0.184,0.009,109.596,0.048913,1,...,0.195652,0.755435,0.074,0.778947,1.0,False,False,True,False,False
45,2020-01-13 10:00:00+00:00,109.848,109.924,109.828,109.849,0.096,0.001,109.876,0.010417,1,...,0.781250,0.208333,0.212,0.803030,1.0,False,False,True,False,False
76,2020-01-20 14:00:00+00:00,110.184,110.192,110.136,110.182,0.056,0.002,110.164,0.035714,-1,...,0.142857,0.821429,0.019,0.452381,1.0,True,True,True,False,False
93,2020-01-23 10:00:00+00:00,109.532,109.652,109.500,109.530,0.152,0.002,109.576,0.013158,-1,...,0.789474,0.197368,0.052,0.382353,-1.0,False,False,True,False,False


In [156]:
df_plot = df.iloc[330:400]
df_markers = df_plot[(df_plot.MARUBOZU==True) | (df_plot.HAMMER == True) | (df_plot.DOJI == True)]

In [157]:
plot_candles(df_plot, df_markers)

Double candle patterns

In [158]:
df_plot = df.iloc[330:400]
df_markers = df_plot[(df_plot.ENGULFING==True)]

In [159]:
plot_candles(df_plot, df_markers)